In [1]:
import requests, os
from dotenv import load_dotenv
from fetch_all_data import registration_data, demographic_data

load_dotenv()

True

In [34]:
print(f"Fetched {len(registration_data)} registration records.")

No more pages to fetch from https://us-central1-boston-family-days---prod.cloudfunctions.net/getRegistrationData
Fetched 10990 registration records.


In [35]:
print(f"Fetched {len(demographic_data)} demographic records.")

No more pages to fetch from https://us-central1-boston-family-days---prod.cloudfunctions.net/getDemographicData
Fetched 8935 demographic records.


In [36]:
import pandas as pd

# Convert the fetched data into DataFrames
registration_df = pd.DataFrame(registration_data)
demographic_df = pd.DataFrame(demographic_data)

# Perform an outer join on the 'passId' column
merged_df = pd.merge(registration_df, demographic_df, on='passId', how='outer')

# Normalize email addresses by trimming spaces and converting to lowercase
merged_df['email'] = merged_df['email'].str.strip().str.lower()

# 1. Distribution of preferred languages (default to 'english' if empty)
merged_df['preferredCommunicationLanguage'] = merged_df['preferredCommunicationLanguage'].fillna('english')
language_counts = merged_df['preferredCommunicationLanguage'].value_counts()
language_percentages = (language_counts / language_counts.sum() * 100).round(1)
language_distribution = pd.DataFrame({
    'Count': language_counts,
    'Percentage': language_percentages
})
print("Preferred Language Distribution:")
print(language_distribution)

# 2. Count and percentage of duplicate students (same first and last name) for different email addresses
duplicate_students = merged_df.groupby(['firstName', 'lastName']).email.nunique()
duplicate_students_count = duplicate_students[duplicate_students > 1].sum()  # Total number of duplicate entries
total_entries = len(merged_df)
duplicate_students_percentage = ((duplicate_students_count * 2 / total_entries) * 100).round(1)
print("\nNumber of Duplicate Student Entries (same first and last name, different emails):")
print(f"Count: {duplicate_students_count}, Percentage: {duplicate_students_percentage:.1f}%")

# 3. Number and percentage of students under the same email addresses
students_per_email = merged_df.groupby('email').size()
email_distribution_counts = students_per_email.value_counts().sort_index()
email_distribution_percentages = (email_distribution_counts / email_distribution_counts.sum() * 100).round(1)
email_distribution = pd.DataFrame({
    'Count': email_distribution_counts,
    'Percentage': email_distribution_percentages
})
print("\nNumber of Students per Email Address:")
print(email_distribution)

Preferred Language Distribution:
                                Count  Percentage
preferredCommunicationLanguage                   
english                         10740        97.7
spanish-latin-american            177         1.6
mandarin                           26         0.2
haitian-creole                     17         0.2
cantonese                           8         0.1
portuguese-brazilian                8         0.1
vietnamese                          7         0.1
cabo-verdean-creole                 3         0.0
french-european                     2         0.0
arabic-standard                     1         0.0
somali                              1         0.0

Number of Duplicate Student Entries (same first and last name, different emails):
Count: 210, Percentage: 3.8%

Number of Students per Email Address:
    Count  Percentage
1    5953        73.1
2    1704        20.9
3     358         4.4
4      92         1.1
5      19         0.2
6       7         0.1
7       2   

In [37]:
# Calculate completeness percentage for each field
def calculate_completeness(df):
    """Calculate the percentage of non-null values for each column in the DataFrame."""
    total_rows = len(df)
    completeness = {}
    
    for column in df.columns:
        non_null_count = df[column].count()  # Count non-null values
        completeness[column] = round((non_null_count / total_rows) * 100, 1)
    
    # Convert to DataFrame and sort by completeness percentage
    completeness_df = pd.DataFrame({
        'Field': completeness.keys(),
        'Completeness (%)': completeness.values()
    }).sort_values('Completeness (%)', ascending=False)
    
    return completeness_df

# Calculate and display completeness for all fields
field_completeness = calculate_completeness(merged_df)
print("\nField Completeness Percentages:")
print(field_completeness)


Field Completeness Percentages:
                             Field  Completeness (%)
0                             id_x             100.0
1                            email             100.0
2                        firstName             100.0
5                         lastName             100.0
6   preferredCommunicationLanguage             100.0
8                      createdAt_x             100.0
10                          passId             100.0
11                          school             100.0
12                          status             100.0
4                   parentLastName              99.9
9                        profileId              99.7
3                  parentFirstName              98.9
26                     createdAt_y              81.3
14                            id_y              81.3
28                       studentId              50.6
20                           grade              48.0
7                      phoneNumber              29.4
18           

In [38]:
# Get unique schools and sort them alphabetically
unique_schools = merged_df['school'].unique()
unique_schools.sort()

# Print the list of unique schools
# print("Unique schools:")
# for school in unique_schools:
#     print(school)

# Alternatively, if you want to see the count of students per school:
school_counts = merged_df['school'].value_counts().sort_index()
print("\nSchool distribution:")
print(school_counts)


School distribution:
school
                                                     133
Academy of the Pacific Rim Charter Public School     507
Bridge Boston Charter School                         330
Brooke Charter School                               2188
Conservatory Lab Charter School                      448
                                                    ... 
xb                                                     1
yeshiva-ohr-yisrael                                    4
ymca                                                   1
ymca-parkway                                           1
young-achievers-k-8-school                             4
Name: count, Length: 536, dtype: int64


In [39]:
import pandas as pd
from thefuzz import fuzz
import re
import json

def normalize_school_name(name):
    """Normalize school name by removing special characters, spaces, and converting to lowercase"""
    if pd.isna(name):
        return ""
    return re.sub(r'[^a-zA-Z0-9]', '', name.lower())

def extract_readable_date(date_str):
    """Extract readable date from JSON string"""
    if pd.isna(date_str):
        return None
    try:
        # Handle both string and dictionary inputs
        if isinstance(date_str, str):
            date_dict = json.loads(date_str.replace("'", '"'))
        elif isinstance(date_str, dict):
            date_dict = date_str
        else:
            return str(date_str)
            
        return date_dict.get('readable')
    except Exception as e:
        # If there's an error, return the original string
        return str(date_str) if date_str else None

def fuzzy_match_schools(df, target_schools, threshold=80):
    """
    Perform fuzzy matching on school names
    threshold: minimum similarity score (0-100) to consider a match
    """
    # Create a working copy of the dataframe
    working_df = df.copy()
    
    # Handle createdAt fields
    working_df['createdAt'] = working_df['createdAt_x'].apply(extract_readable_date)
    working_df['createdAt'] = working_df.apply(
        lambda x: x['createdAt'] if pd.notna(x['createdAt']) 
        else extract_readable_date(x['createdAt_y']), axis=1
    )
    
    # Drop the original createdAt columns
    working_df = working_df.drop(['createdAt_x', 'createdAt_y'], axis=1)
    
    # Normalize target schools
    normalized_targets = [normalize_school_name(school) for school in target_schools]
    target_map = dict(zip(normalized_targets, target_schools))
    
    # Function to find best match
    def find_best_match(school_name):
        normalized_name = normalize_school_name(school_name)
        best_score = 0
        best_match = None
        
        for target in normalized_targets:
            score = fuzz.ratio(normalized_name, target)
            if score > best_score and score >= threshold:
                best_score = score
                best_match = target_map[target]
        
        return best_match if best_match else None

    # Add matched school column
    working_df['matched_school'] = working_df['school'].apply(find_best_match)
    
    # Filter rows where we found a match
    matched_df = working_df[working_df['matched_school'].notna()].copy()
    
    return matched_df

# List of target schools
target_schools = [
    "Torah Academy",
    "Yeshiva Ohr Yisrael"
]

# Perform fuzzy matching
matched_data = fuzzy_match_schools(merged_df, target_schools)

# Export to CSV
matched_data.to_csv('catholic_schools_data.csv', index=False)

# Print summary
print(f"Found {len(matched_data)} matching records")
print("\nMatched schools distribution:")
print(matched_data['matched_school'].value_counts())

Found 36 matching records

Matched schools distribution:
matched_school
Torah Academy          32
Yeshiva Ohr Yisrael     4
Name: count, dtype: int64


In [40]:
# Print all school counts to txt file
with open('school_counts.txt', 'w') as f:
    f.write(school_counts.to_string())
    